In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import pandas as pd

import kraft

In [3]:
gene_x_cell_line = pd.read_csv(
    "/media/kwat/CarrotCake/garden/data/cancer_cell_line/protein/gene_x_cell_line.tsv", sep="\t", index_col=0
)

In [4]:
for sea_method in ("classic",):

    print(sea_method)

    gene_set_x_cell_line = pd.read_csv(
        "/media/kwat/CarrotCake/garden/data/cancer_cell_line/gene_set/gene_set_x_sample_x_{}.tsv".format(sea_method),
        sep="\t",
        index_col=0,
    )

    gene_sets = gene_set_x_cell_line.index.to_numpy()

    gene_to_gene_sets = {}

    gene_to_n_gene_set = {}

    for gene in gene_x_cell_line.index:

        _gene_ = "_{}_".format(gene)

        gene_sets_matched = tuple(
            gene_set for gene_set in gene_sets if _gene_ in gene_set
        )

        if 0 < len(gene_sets_matched):

            gene_to_gene_sets[gene] = gene_sets_matched

            gene_to_n_gene_set[gene] = len(gene_sets_matched)

    gene_to_n_gene_set = pd.Series(gene_to_n_gene_set)

    gene_to_n_gene_set = gene_to_n_gene_set.sort_values(ascending=False)

    print(gene_to_n_gene_set)

    gene_x_cell_line = gene_x_cell_line.loc[gene_to_n_gene_set.index, :]

    print(gene_x_cell_line.shape)

    scores_directory_path = "{}.scores/".format(sea_method)

    kraft.path(scores_directory_path)

    for gene, cell_line_values in gene_x_cell_line.iterrows():

        directory_path = "{}/{}/".format(scores_directory_path, gene)

        if not os.path.isdir(directory_path):

            kraft.path(directory_path)

            kraft.function_heat_map(
                cell_line_values,
                gene_set_x_cell_line,
                kraft.get_ic,
                n_job=8,
                n_sampling=0,
                n_permutation=0,
                plot=False,
                directory_path=directory_path,
            )

classic


In [ ]:
gene_x_cell_line = gene_x_cell_line.loc[
    50 < (~gene_x_cell_line.isna()).sum(axis=1), :
]

gene_x_cell_line

In [ ]:
gene_x_sea_method = pd.DataFrame(
    index=gene_x_cell_line.index, columns=("classic",)
)

for sea_method in gene_x_sea_method.columns:

    for gene in gene_x_sea_method.index:

        scores = (
            pd.read_table(
                "{}.scores/{}/scores.tsv".format(sea_method, gene),
                sep="\t",
                index_col=0,
            )
            .loc[:, "Score"]
        )
        
        scores = scores.loc[(1/3) < scores][:1000]

        gene_x_sea_method.loc[gene, sea_method] = scores.loc[
            scores.index & gene_to_gene_sets[gene]
        ].median()

gene_x_sea_method

In [ ]:
n_extreme = 5

for sea_method, gene_scores in gene_x_sea_method.items():

    for gene in gene_scores.dropna().sort_values().iloc[-3:].index:

        kraft.function_heat_map(
            gene_x_cell_line.loc[gene, :],
            gene_set_x_cell_line.loc[gene_to_gene_sets[gene], :],
            kraft.get_ic,
            n_sampling=0,
            n_permutation=0,
            layout={"title": {"text": sea_method}},
        )

In [ ]:
kraft.plot_plotly(
    {
        "layout": {"yaxis": {"title": {"text": "|IC| Median"}}},
        "data": [
            {
                "type": "box",
                "showlegend": False,
                "x": (sea_method.title(),) * gene_scores.size,
                "y": gene_scores,
                "text": gene_scores.index,
                "boxpoints": "all",
                "jitter": 1,
            }
            for sea_method, gene_scores in gene_x_sea_method.items()
        ],
    }
)

In [ ]:
gene_changes = gene_x_sea_method.loc[:, "2020"] - gene_x_sea_method.loc[:, "classic"]

kraft.plot_histogram((gene_changes,), layout={"xaxis": {"title": {"text": "Change"}}})